In [1]:
import pandas as pd
import numpy as np
import folium 
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../Datas/Filtered Data For Neigborhoods.csv")
# read usneighborhoods.csv
zf = pd.read_csv("../Datas/usneighborhoods.csv")

In [3]:
x = ["MA", 'IL', 'CA', 'NY', 'TX', 'FL', 'CO']
zf = zf[zf['state_id'].isin(x)]

In [4]:
df['period_end'] = pd.to_datetime(df['period_end'])
# Group by 'city' and 'region', then sort each group by 'period_end'
sorted_df = df.groupby(['city', 'region'], group_keys=False).apply(lambda x: x.sort_values('period_end'))
# Group by 'city' and 'region', then get the last row from each group
last_rows = sorted_df.groupby(['city', 'region']).last()
df = last_rows.reset_index()

In [5]:
cities_to_filter = [
    "Boston",
    "Chicago",
    "San Francisco",
    "New York",
    "Austin",
    "Miami",
    "Denver"
]

# Filter the DataFrame based on the list of cities
filtered_df = df[df["city"].isin(cities_to_filter)]

In [6]:
# Filter the DataFrame based on the list of cities
filtered_zf = zf[zf["city_name"].isin(cities_to_filter)]

In [7]:
filtered_df['neighborhood'] = filtered_df['region'].str.split("-").str[1]

In [8]:
filtered_df['city'] = filtered_df['city'].str.strip()
# ['neighborhood'] STRİP
filtered_df['neighborhood'] = filtered_df['neighborhood'].str.strip()

# filtered_zf same
filtered_zf['city_name'] = filtered_zf['city_name'].str.strip()
filtered_zf['neighborhood'] = filtered_zf['neighborhood'].str.strip()

In [9]:
filtered_df['city_neig'] = filtered_df['city'] + ' '+ filtered_df['neighborhood']

In [10]:
filtered_zf['city_neig'] =  filtered_zf['city_name'] + ' ' +filtered_zf['neighborhood'];

In [11]:
# drop neighborhood_ascii city_id	state_name	state_id	source	timezone	zips	county_fips	county_name id FROM from filtered_zf

# List of columns to drop
columns_to_drop = ["neighborhood_ascii", "city_id", "state_name", "state_id", "source", "timezone", "zips", "county_fips", "county_name", "id"]

# Drop the specified columns
filtered_zf = filtered_zf.drop(columns=columns_to_drop)

In [12]:
filtered_zf['city_neig'] = filtered_zf['city_neig'].str.title()

In [13]:
filtered_df['city_neig'] = filtered_df['city_neig'].str.title()

In [14]:
filtered_df = filtered_df.merge(filtered_zf, on='city_neig',how="left")

In [15]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

# Your Google Maps API key
google_maps_api_key = os.getenv("GOOGLE_API_KEY")

def get_lat_lon_google_maps(address):
    try:
        response = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={google_maps_api_key}")
        data = response.json()
        print(data,"data")
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            lat = location['lat']
            lon = location['lng']
            return lat, lon, data 
        else:
            return None, None, data 
    except:
        return None, None

In [16]:
filtered_df_na = filtered_df[filtered_df['lat'].isna()]

In [17]:
filtered_df = filtered_df[filtered_df['lat'].notna()]

In [18]:
filtered_df_na['geocode_response'] = ''

In [20]:
from tqdm import tqdm

In [27]:

# Apply geocoding function directly to the DataFrame
# filtered_df_na[["lat", "lng", "geocode_response"]] = filtered_df_na["region"].progress_apply(lambda address: get_lat_lon_google_maps(address)).apply(pd.Series)
# filtered_df_na[["lat", "lng", "geocode_response"]] = filtered_df_na["region"].apply(lambda address: get_lat_lon_google_maps(address)).progress_apply(pd.Series)


# # Create an empty DataFrame to store the geocode results
# geocode_results = pd.DataFrame(columns=["lat", "lng", "geocode_response"])

# # Use tqdm to track the progress of the iteration
# for address in tqdm(filtered_df_na["region"]):
#     result = get_lat_lon_google_maps(address)
#     # geocode_results = geocode_results.append(result, ignore_index=True
#     geocode_results = pd.concat([geocode_results, pd.DataFrame(result, index=[0])], ignore_index=True)

# # Assign the geocode_results to the original DataFrame
# filtered_df_na[["lat", "lng", "geocode_response"]] = geocode_results

# Create empty lists to store the geocode results
lat_list = []
lng_list = []
geocode_response_list = []

# Use tqdm to track the progress of the iteration
for address in tqdm(filtered_df_na["region"]):
    result = get_lat_lon_google_maps(address)
    lat_list.append(result[0])
    lng_list.append(result[1])
    geocode_response_list.append(result[2])

# Assign the geocode results to the original DataFrame
filtered_df_na["lat"] = np.array(lat_list)
filtered_df_na["lng"] = np.array(lng_list)
filtered_df_na["geocode_response"] = np.array(geocode_response_list)

  0%|          | 1/756 [00:00<02:59,  4.20it/s]

{'results': [{'address_components': [{'long_name': 'Second Street District', 'short_name': 'Second Street District', 'types': ['neighborhood', 'political']}, {'long_name': 'Austin', 'short_name': 'Austin', 'types': ['locality', 'political']}, {'long_name': 'Travis County', 'short_name': 'Travis County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Texas', 'short_name': 'TX', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '78701', 'short_name': '78701', 'types': ['postal_code']}], 'formatted_address': 'Second Street District, Austin, TX 78701, USA', 'geometry': {'bounds': {'northeast': {'lat': 30.2665455, 'lng': -97.7445379}, 'southwest': {'lat': 30.2629454, 'lng': -97.7492506}}, 'location': {'lat': 30.2649178, 'lng': -97.74642349999999}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 30.2665455, 'lng': -97.7445379}, 'southwest': {'la

  0%|          | 3/756 [00:00<02:51,  4.38it/s]

{'results': [{'address_components': [{'long_name': 'Anderson Mill West', 'short_name': 'Anderson Mill West', 'types': ['neighborhood', 'political']}, {'long_name': 'Austin', 'short_name': 'Austin', 'types': ['locality', 'political']}, {'long_name': 'Williamson County', 'short_name': 'Williamson County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Texas', 'short_name': 'TX', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Anderson Mill West, Austin, TX, USA', 'geometry': {'bounds': {'northeast': {'lat': 30.469826, 'lng': -97.81771599999999}, 'southwest': {'lat': 30.4555339, 'lng': -97.846101}}, 'location': {'lat': 30.4611355, 'lng': -97.8299965}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 30.469826, 'lng': -97.81771599999999}, 'southwest': {'lat': 30.4555339, 'lng': -97.846101}}}, 'place_id': 'ChIJLbVaBmMyW4YRIEiFqs68NIY'

  1%|          | 4/756 [00:00<02:56,  4.26it/s]

...

{'results': [{'address_components': [{'long_name': 'San Francisco', 'short_name': 'SF', 'types': ['locality', 'political']}, {'long_name': 'San Francisco County', 'short_name': 'San Francisco County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'San Francisco, CA, USA', 'geometry': {'bounds': {'northeast': {'lat': 37.9298239, 'lng': -122.28178}, 'southwest': {'lat': 37.6398299, 'lng': -123.1328309}}, 'location': {'lat': 37.7749295, 'lng': -122.4194155}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 37.812, 'lng': -122.3482}, 'southwest': {'lat': 37.70339999999999, 'lng': -122.527}}}, 'place_id': 'ChIJIQBpAG2ahYAR_6128GcTUEo', 'types': ['locality', 'political']}], 'status': 'OK'} data
{'results': [{'address_components': [{'long_name': 'San Franci

100%|█████████▉| 755/756 [02:13<00:00,  5.90it/s]

{'results': [{'address_components': [{'long_name': 'West Portal', 'short_name': 'West Portal', 'types': ['neighborhood', 'political']}, {'long_name': 'San Francisco', 'short_name': 'SF', 'types': ['locality', 'political']}, {'long_name': 'San Francisco County', 'short_name': 'San Francisco County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'West Portal, San Francisco, CA, USA', 'geometry': {'bounds': {'northeast': {'lat': 37.7437493, 'lng': -122.463686}, 'southwest': {'lat': 37.7346256, 'lng': -122.4756444}}, 'location': {'lat': 37.7395735, 'lng': -122.470007}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 37.7437493, 'lng': -122.463686}, 'southwest': {'lat': 37.7346256, 'lng': -122.4756444}}}, 'partial_match': True, 'place_id': 'ChIJZzTFa-p9j

100%|██████████| 756/756 [02:14<00:00,  5.64it/s]

{'results': [{'address_components': [{'long_name': 'Yerba Buena', 'short_name': 'Yerba Buena', 'types': ['neighborhood', 'political']}, {'long_name': 'San Francisco', 'short_name': 'SF', 'types': ['locality', 'political']}, {'long_name': 'San Francisco County', 'short_name': 'San Francisco County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Yerba Buena, San Francisco, CA, USA', 'geometry': {'bounds': {'northeast': {'lat': 37.7892677, 'lng': -122.3951604}, 'southwest': {'lat': 37.779049, 'lng': -122.4080933}}, 'location': {'lat': 37.7850045, 'lng': -122.4004816}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 37.7892677, 'lng': -122.3951604}, 'southwest': {'lat': 37.779049, 'lng': -122.4080933}}}, 'place_id': 'ChIJ58VAPnyAhYARkBzVL7yn1Ck', 'type

In [28]:
filtered_df_na

,city,region,period_end,new_listings,homes_sold,off_market_in_two_weeks,median_sale_price,median_dom,months_of_supply,state_code,neighborhood_x,city_neig,neighborhood_y,lat,lng,city_name,geocode_response
0,Austin,"Austin, TX - 2nd Street District",2023-08-31,5.0,4.0,0.000000,2102500.0,123.5,NaN,TX,2nd Street District,Austin 2Nd Street District,NaN,30.264918,-97.746423,NaN,{'results': [{'address_components': [{'long_na...
1,Austin,"Austin, TX - Allandale",2023-08-31,45.0,24.0,0.300000,784525.0,34.5,NaN,TX,Allandale,Austin Allandale,NaN,30.339325,-97.746952,NaN,{'results': [{'address_components': [{'long_na...
2,Austin,"Austin, TX - Anderson Mill West",2023-08-31,8.0,12.0,0.000000,445000.0,36.0,NaN,TX,Anderson Mill West,Austin Anderson Mill West,NaN,30.461136,-97.829996,NaN,{'results': [{'address_components': [{'long_na...
3,Austin,"Austin, TX - Angus Valley",2023-08-31,13.0,9.0,0.333333,799000.0,51.0,NaN,TX,Angus Valley,Austin Angus Valley,NaN,30.423717,-97.737596,NaN,{'results': [{'address_components': [{'long_na...
4,Austin,"Austin, TX - Arts District",2023-08-31,11.0,3.0,0.000000,247500.0,46.0,NaN,TX,Arts District,Austin Arts District,NaN,30.267153,-97.743061,NaN,{'results': [{'address_components': [{'long_na...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,San Francisco,"San Francisco, CA - Southwest San Francisco",2023-08-31,444.0,399.0,0.327869,1450000.0,21.0,NaN,CA,Southwest San Francisco,San Francisco Southwest San Francisco,NaN,37.774929,-122.419415,NaN,{'results': [{'address_components': [{'long_na...
1249,San Francisco,"San Francisco, CA - Transmission",2023-08-31,4.0,1.0,0.000000,1249000.0,26.0,NaN,CA,Transmission,San Francisco Transmission,NaN,37.774929,-122.419415,NaN,{'results': [{'address_components': [{'long_na...
1255,San Francisco,"San Francisco, CA - West Portal Park",2023-08-31,2.0,2.0,1.000000,2300000.0,10.0,NaN,CA,West Portal Park,San Francisco West Portal Park,NaN,37.739573,-122.470007,NaN,{'results': [{'address_components': [{'long_na...
1257,San Francisco,"San Francisco, CA - Western South of Market",2023-08-31,36.0,19.0,0.142857,712000.0,70.0,NaN,CA,Western South of Market,San Francisco Western South Of Market,NaN,37.785025,-122.400500,NaN,{'results': [{'address_components': [{'long_na...


In [29]:
df = pd.concat([filtered_df,filtered_df_na], axis=0)

In [30]:
# drop city_name neighborhood_y from df
df.drop(['city_name', 'neighborhood_y'], axis=1, inplace=True)

In [31]:
df.to_csv("../Datas/Last Shape of Data.csv",index=False)